In [26]:
import time
import math
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import dataset
import cv2

from sklearn.metrics import confusion_matrix
from datetime import timedelta

import tensorflow as tf
import dataset

import os
import warnings
warnings.filterwarnings('ignore')

In [27]:
num_channels = 3
img_size = 244
img_size_flat = img_size * img_size * num_channels

img_shape = (img_size, img_size)

classes = ['empty', 'full']
num_classes = len(classes)

batch_size = 32

validation_size = .2

train_path = './dataset/'
test_path = './test/'
checkpoint_dir = './models/'

In [28]:
image_shape = [None, img_size, img_size, num_channels]

In [29]:
data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)
test_images, test_ids = dataset.read_test_set(test_path, img_size)

Reading training images
Loading empty files (Index: 0)
Loading full files (Index: 1)
Reading test images


In [30]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(test_images)))
print("- Validation-set:\t{}".format(len(data.valid.labels)))

Size of:
- Training-set:		2117
- Test-set:		5
- Validation-set:	529


In [ ]:
def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat #, num_features

## MobileNetV2

In [ ]:
def _conv_block(inputs, filters, kernel, strides):
    
    x = tf.nn.conv2d(filters, kernel, padding='same', strides=strides)(inputs)
    x = tf.nn.batch_normalization()(x)
    return relu6(x)


def _bottleneck(inputs, filters, kernel, t, s, r=False):
    
    tchannel = inputs.shape[-1] * t
    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))
    x = tf.nn.depthwise_conv2d(kernel, strides=(s, s), padding='same')(x)
    x = tf.nn.batch_normalization()(x)
    x = tf.nn.relu6(x)
    
    x = tf.nn.conv2d(filters, (1, 1), strides=(1, 1), padding='same')(x)
    x = tf.nn.batch_normalization()(x)

    if r:
        x = x = tf.math.add(x, inputs)
    return x


def _inverted_residual_block(inputs, filters, kernel, t, strides, n):
    
    x = _bottleneck(inputs, filters, kernel, t, strides)
    
    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, 1, True)

    return x

def MobileNetV2(inputs, plot_model=False):
    """MobileNetv2
    This function defines a MobileNetv2 architecture.
    # Arguments
        input_shape: An integer or tuple/list of 3 integers, shape
            of input tensor.
        k: Integer, number of classes.
        plot_model: Boolean, whether to plot model architecture or not
    # Returns
        MobileNetv2 model.
    """

#     x = tf.placeholder("float", image_shape)
#     inputs = Input(shape=input_shape, name='input')
    x = _conv_block(inputs, 32, (3, 3), strides=(2, 2))

    x = _inverted_residual_block(x, 16, (3, 3), t=1, strides=1, n=1)
    x = _inverted_residual_block(x, 24, (3, 3), t=6, strides=2, n=2)
    x = _inverted_residual_block(x, 32, (3, 3), t=6, strides=2, n=3)
    x = _inverted_residual_block(x, 64, (3, 3), t=6, strides=2, n=4)
    x = _inverted_residual_block(x, 96, (3, 3), t=6, strides=1, n=3)
    x = _inverted_residual_block(x, 160, (3, 3), t=6, strides=2, n=3)
    x = _inverted_residual_block(x, 320, (3, 3), t=6, strides=1, n=1)

    x = _conv_block(x, 1280, (1, 1), strides=(1, 1))
#     x = flatten_layer(x)
    x = tf.reshape((1, 1, 1280))(x)
    x = tf.nn.dropout(0.3, name='Dropout')(x)
    x = tf.nn.conv2d(k, (1, 1), padding='same')(x)
    output = tf.nn.softmax('softmax', name='final_activation')(x)
#     output = tf.reshape((k,), name='output')(x)
    
#     model = tf.keras.models.Model(inputs, output)
#     model.summary()
#     if plot_model:
#         tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

    return output

def trashcan_net(inputs, plot_model=False):
    
    x = MobileNetV2(inputs, plot_model=False)
    x = tf.nn.avg_pool(ksize=(7, 7))(x)
    x = flatten_layer(x)
    x = tf.compat.v1.layers.dense(128)(x)
    x = tf.nn.relu6(x)
    x = tf.nn.dropout(0.5)(x)
    x = tf.compat.v1.layers.dense(2)(x)
    output = tf.nn.softmax()(x)
    
    return output

In [40]:
# Итерируем по эпохам
grad_log = []

epochs = 10  # учим немного, т.к. задача посмотреть , что происходит с градиентом

for epoch in range(epochs): 
    print(f'\nНачинаем эпоху {epoch}') 

    # Итерируем по батчам в датасете
    for step in range(epochs): 
        
        # Откроем GradientTape чтобы записать операции
        # выполняемые во время прямого прохода, включающего автодифференцирование
        with tf.GradientTape() as tape: 
            # Запустим прямой проход слоя
            
            x_batch, y_true_batch, _, cls_batch = data.train.next_batch(step+1)
            
            print(x_batch)
            
            preds = trashcan_net(x_batch) 

            # Вычислим значение потерь для этого батча
            loss_value = loss_fn(y_true_batch, preds)

            # Используем gradient tape для автоматического извлечения градиентов 
            # обучаемых переменных относительно потерь
            grads = tape.gradient(loss_value, modelg.trainable_weights) 
            g_g = []

            # пишем логи для сохранения значений градиента и веса по одной цепи 
            for g_s in grads:
                # допишем логи значений градиента в зависимости от размера тензора градиента
                # if len(g_s.numpy().shape) == 1:
                #     g_g.append(g_s.numpy()[0])
                if len(g_s.numpy().shape) == 2:
                    g_g.append(g_s.numpy()[0, 0]) 

                
        # добавляем текущие логи по слоям к общей записи
        grad_log.append(g_g)

        # Выполним один шаг градиентного спуска,
        # обновив значение переменных минимизирующих потери
        optimizer.apply_gradients(zip(grads, modelg.trainable_weights)) 

        # Пишем лог каждые 200 шагов
        if step % 200 == 0:
            print(f'Эпоха {epoch + 1}/{epochs}', end='. ')
            print(f'Шаг {step}. Лосс на обучении (для одного батча) на шаге: {loss_value}') 
            print(f'Уже увидели: {(step + 1) * batch_size} примеров')


Начинаем эпоху 0
[[[[0.07450981 0.08627451 0.10196079]
   [0.27450982 0.28627452 0.3019608 ]
   [0.7019608  0.70980394 0.7254902 ]
   ...
   [0.909804   0.74509805 0.5803922 ]
   [0.91372555 0.7372549  0.65882355]
   [0.41960788 0.26666668 0.37254903]]

  [[0.07450981 0.08627451 0.10196079]
   [0.24313727 0.25490198 0.27058825]
   [0.6862745  0.69803923 0.7137255 ]
   ...
   [0.73333335 0.5137255  0.25490198]
   [0.6313726  0.43921572 0.15294118]
   [0.70980394 0.5568628  0.37647063]]

  [[0.06666667 0.08235294 0.09411766]
   [0.227451   0.2392157  0.2509804 ]
   [0.68235296 0.69411767 0.70980394]
   ...
   [0.89019614 0.6666667  0.3803922 ]
   [0.8941177  0.70980394 0.34117648]
   [0.7411765  0.6117647  0.28235295]]

  ...

  [[0.7803922  0.80392164 0.8313726 ]
   [0.53333336 0.5568628  0.58431375]
   [0.6        0.62352943 0.6509804 ]
   ...
   [0.14901961 0.21176472 0.27450982]
   [0.15686275 0.20392159 0.27058825]
   [0.15294118 0.19215688 0.25882354]]

  [[0.48235297 0.5058824  0

AttributeError: 'int' object has no attribute 'shape'

In [8]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

In [9]:
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

In [10]:
x

<tf.Tensor 'x:0' shape=(?, 178608) dtype=float32>

In [11]:
x_image

<tf.Tensor 'Reshape:0' shape=(?, 244, 244, 3) dtype=float32>

In [12]:
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

In [13]:
y_true_cls = tf.argmax(y_true, axis=1)

In [14]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)

In [15]:
layer_conv1, weights_conv1

(<tf.Tensor 'Relu:0' shape=(?, 122, 122, 32) dtype=float32>,
 <tf.Variable 'Variable:0' shape=(3, 3, 3, 32) dtype=float32_ref>)

In [16]:
layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

In [17]:
layer_conv2, weights_conv2

(<tf.Tensor 'Relu_1:0' shape=(?, 61, 61, 32) dtype=float32>,
 <tf.Variable 'Variable_2:0' shape=(3, 3, 32, 32) dtype=float32_ref>)

In [18]:
layer_conv3, weights_conv3 = \
    new_conv_layer(input=layer_conv2,
                   num_input_channels=num_filters2,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=True)

In [19]:
layer_conv3, weights_conv3

(<tf.Tensor 'Relu_2:0' shape=(?, 31, 31, 64) dtype=float32>,
 <tf.Variable 'Variable_4:0' shape=(3, 3, 32, 64) dtype=float32_ref>)

In [20]:
layer_flat, num_features = flatten_layer(layer_conv3)

In [21]:
layer_flat, num_features

(<tf.Tensor 'Reshape_1:0' shape=(?, 61504) dtype=float32>, 61504)

In [22]:
num_features

61504

In [23]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

In [24]:
layer_fc1

<tf.Tensor 'Relu_3:0' shape=(?, 128) dtype=float32>

In [25]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)

In [26]:
layer_fc2

<tf.Tensor 'add_4:0' shape=(?, 2) dtype=float32>

In [27]:
y_pred = tf.nn.softmax(layer_fc2)

In [28]:
y_pred

<tf.Tensor 'Softmax:0' shape=(?, 2) dtype=float32>

In [29]:
y_pred_cls = tf.argmax(y_pred, axis=1)

In [30]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2, labels=y_true)

In [31]:
cost = tf.reduce_mean(cross_entropy)

In [32]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [33]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [34]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [35]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [36]:
train_batch_size = batch_size

In [37]:
def print_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    # Calculate the accuracy on the training-set.
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%}, Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

In [38]:
# Counter for total number of iterations performed so far.
total_iterations = 0

def optimize(num_iterations):
    # Ensure we update the global variable rather than a local copy.
    global total_iterations

    # Start-time used for printing time-usage below.
    start_time = time.time()
    
    best_val_loss = float("inf")
    patience = 0

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(train_batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(train_batch_size)

        # Convert shape from [num examples, rows, columns, depth]
        # to [num examples, flattened image shape]

        x_batch = x_batch.reshape(train_batch_size, img_size_flat)
#         print(x_batch.size)
        x_valid_batch = x_valid_batch.reshape(train_batch_size, img_size_flat)

        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}
        
        feed_dict_validate = {x: x_valid_batch,
                              y_true: y_valid_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)
        

        # Print status at end of each epoch (defined as full pass through training dataset).
        if i % int(data.train.num_examples/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_validate)
            epoch = int(i / int(data.train.num_examples/batch_size))
            
            print_progress(epoch, feed_dict_train, feed_dict_validate, val_loss)
            
            if early_stopping:    
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    patience = 0
                else:
                    patience += 1

                if patience == early_stopping:
                    break

    # Update the total number of iterations performed.
    total_iterations += num_iterations

    # Ending time.
    end_time = time.time()

    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time elapsed: " + str(timedelta(seconds=int(round(time_dif)))))

In [39]:
def plot_example_errors(cls_pred, correct):
    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # correct is a boolean array whether the predicted class
    # is equal to the true class for each image in the test-set.

    # Negate the boolean array.
    incorrect = (correct == False)
    
    # Get the images from the test-set that have been
    # incorrectly classified.
    images = data.valid.images[incorrect]
    
    # Get the predicted classes for those images.
    cls_pred = cls_pred[incorrect]

    # Get the true classes for those images.
    cls_true = data.valid.cls[incorrect]

    # Plot the first 9 images.
    plot_images(images=images, cls_true=cls_true, cls_pred=cls_pred)

In [40]:
def plot_confusion_matrix(cls_pred):
    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # Get the true classifications for the test-set.
    cls_true = data.valid.cls
    
    # Get the confusion matrix using sklearn.
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    # Print the confusion matrix as text.
    print(cm)

    # Plot the confusion matrix as an image.
    plt.matshow(cm)

    # Make various adjustments to the plot.
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [41]:
def print_validation_accuracy(show_example_errors=False, show_confusion_matrix=False):

    # Number of images in the test-set.
    num_test = len(data.valid.images)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # Now calculate the predicted classes for the batches.
    # We will just iterate through all the batches.
    # There might be a more clever and Pythonic way of doing this.

    # The starting index for the next batch is denoted i.
    i = 0

    while i < num_test:
        # The ending index for the next batch is denoted j.
        j = min(i + batch_size, num_test)
        
        print(data.valid.images[i:j, :].size)

        # Get the images from the test-set between index i and j.
        images = data.valid.images[i:j, :].reshape(batch_size, img_size_flat)
        
        

        # Get the associated labels.
        labels = data.valid.labels[i:j, :]

        # Create a feed-dict with these images and labels.
        feed_dict = {x: images,
                     y_true: labels}

        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    cls_true = np.array(data.valid.cls)
    cls_pred = np.array([classes[x] for x in cls_pred]) 

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    # Calculate the number of correctly classified images.
    # When summing a boolean array, False means 0 and True means 1.
    correct_sum = correct.sum()

    # Classification accuracy is the number of correctly classified
    # images divided by the total number of images in the test-set.
    acc = float(correct_sum) / num_test

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

    # Plot some examples of mis-classifications, if desired.
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    # Plot the confusion matrix, if desired.
    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

In [42]:
len(data.valid.images)

793

In [43]:
optimize(num_iterations=1)
print_validation_accuracy(show_example_errors=True)

Epoch 1 --- Training Accuracy:  68.8%, Validation Accuracy:  59.4%, Validation Loss: 0.811
Time elapsed: 0:00:05
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
5715456
4465200


ValueError: cannot reshape array of size 4465200 into shape (32,178608)

In [44]:
optimize(num_iterations=99)

Epoch 2 --- Training Accuracy:  81.2%, Validation Accuracy:  71.9%, Validation Loss: 0.619
Time elapsed: 0:00:03


In [ ]:
print_validation_accuracy(show_example_errors=True)